test3　　演算(t,matmul,sigmoid,relu)のテスト

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [3]:
%cd "ctorch210701/MiniCTorch_Prototype"

/content/drive/My Drive/Colab Notebooks/ctorch210701/MiniCTorch_Prototype


In [4]:
! pip install lark-parser

     |████████████████████████████████| 235kB 14.4MB/s 
  Created wheel for lark-parser: filename=lark_parser-0.11.3-py2.py3-none-any.whl size=99739 sha256=ae8d7b26a38999b7ba2a639861c842afd9b399c488a8d34ab7d6a7af66920580
  Stored in directory: /root/.cache/pip/wheels/81/4d/b0/a94848d29b8ea206b0ef798ad241e6ebbcbc51825bec7832d3
Successfully built lark-parser


In [5]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch
import minictorch.generator as GN
import minictorch.converter as CV

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.t = torch.tensor(np.array([[5.0,6,7], [8,9,10]]))
        #self.b = torch.tensor([1.0,2.,3.],dtype=torch.float64)

    def forward(self, x):
        tt=torch.t(self.t)
        z1=torch.matmul(x,tt)
        #z2=torch.sigmoid(z1)
        z2=torch.relu(z1)
        #z2=torch.tanh(z1)
        #print(z1)
        #print(z2)
        return z2

In [7]:
def generate_json( project, input_x, json_path, check=1 ):

    model = Net()
    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_x, json_path )
    
    if check > 0:
        y2 = model( input_x )
        print(y2)
        y2.sum().backward()
        print( input_to_model.grad )

    return model

In [8]:
def convert_json( project, model, input_x, json_path, rand_flag=0 ):

    folder = "src"
    cpp_fname   = project + ".cpp"
    param_fname = project + "_param.cpp"  # 210719 mod
    cpp_path    = folder + "/" + cpp_fname
    param_path  = folder + "/" + param_fname
    make_path   = folder + "/" + "Makefile"

    # load json file
    print( "[JSON]", json_path )
    fp = open( json_path )
    obj = json.load( fp )

    # save parameter file
    code1 = CV.c_param_generator( obj, model, input_x )
    if len(code1) > 0:
       print( "[PARAM]", param_path )
       ofparam = open( param_path, "w" )
       ofparam.write( code1 )
    else:
       param_fname=""

    # save cpp file
    print( "[CPP]  ", cpp_path )
    code2 = CV.c_code_generator( obj, model, rand_flag )  # 210719 mod

    #print("[SAVE]",args.path+"/"+args.output)
    #ofp= open( args.path + "/" + args.output, "w" )
    ofp = open( cpp_path, "w" )
    ofp.write( code2 )

    # save make file
    print( "[MAKE] ", make_path )
    make_code = CV.makefile_generator( project )  # 210719 mod

    #makefp= open( args.path + "/" + "Makefile", "w" )
    makefp = open( make_path, "w" )
    makefp.write( make_code )

In [9]:
input_to_model = torch.tensor(np.array([[1.0, 2, 3]]), requires_grad=True)

project = "test3"
json_fname = project + ".json"
json_path  = "network/" + json_fname

model = generate_json( project, input_to_model, json_path )

[JSON] network/test3.json
tensor([[38., 56.]], dtype=torch.float64, grad_fn=<ReluBackward0>)
tensor([[13., 15., 17.]], dtype=torch.float64)


In [10]:
convert_json( project, model, input_to_model, json_path )

[JSON] network/test3.json
[PARAM] src/test3_param.cpp
[CPP]   src/test3.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'shape': [1, 3], 'out': [3], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'shape': [2, 3], 'constant_value': [5.0, 6.0, 7.0, 8.0, 9.0, 10.0], 'out': [2], 'sorted_id': 1}
{'name': 'Net/tt', 'op': 'aten::t', 'in': [1], 'shape': [3, 2], 'out': [3], 'sorted_id': 2}
{'name': 'Net/z1', 'op': 'aten::matmul', 'in': [0, 2], 'shape': [1, 2], 'out': [4], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'aten::relu', 'in': [3], 'shape': [1, 2], 'out': [5], 'sorted_id': 4}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [4], 'shape': [1, 2], 'out': [], 'sorted_id': 5}
[MAKE]  src/Makefile


In [11]:
!g++ -std=c++14 ./src/test3.cpp ./src/test3_param.cpp -I ../../ctorch/lib -lcblas -o test3

(注意)  ctorch/libにxtensor関連のincludeを置いています。

In [12]:
!./test3

### forward computation ...
{{ 38.,  56.}}
### backward computation ...
{{ 13.,  15.,  17.}}


In [ ]:
print(torch.__version__)

1.9.0+cu102


In [ ]:
!python -V

Python 3.7.10
